In [1]:
import pandas as pd
import json
import pickle
import os

In [2]:
dt = '20240713'
version = 'v1'

output_dir = os.path.join('outputs', f'{version}_{dt}')
os.makedirs(output_dir, exist_ok=True)

# 加载文档片段

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/2024全球经济金融展望报告.pdf")
documents = loader.load()

In [4]:
from uuid import uuid4
import os
import pickle
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, filepath, chunk_size=400, chunk_overlap=40, seperators=['\n\n\n', '\n\n'], force_split=False):
    if os.path.exists(filepath) and not force_split:
        print('found cache, restoring...')
        return pickle.load(open(filepath, 'rb'))

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=seperators
    )
    split_docs = splitter.split_documents(documents)
    for chunk in split_docs:
        chunk.metadata['uuid'] = str(uuid4())

    pickle.dump(split_docs, open(filepath, 'wb'))

    return split_docs

splitted_docs = split_docs(documents, os.path.join(output_dir, 'split_docs.pkl'), chunk_size=500, chunk_overlap=50)

found cache, restoring...


In [5]:
uuid2doc = {doc.metadata['uuid']: doc for doc in splitted_docs}

In [6]:
len(uuid2doc)

52

# 加载抽取的QA

In [7]:
qa_df = pd.read_excel(os.path.join(output_dir, f'question_answer.xlsx'))
qa_df = qa_df[qa_df['dataset'] == 'train']

In [8]:
len(qa_df)

290

In [9]:
qa_df.head(3)

,uuid,question,answer,context,doc,qa_type,score,score_reason,dataset
4,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,美元指数在2023年的走势如何？,美元指数高位震荡后走弱。,美元指数高位震荡后走弱,研究院\n全球经济金融展望报告\n要点2024年年报（总第57期） 报告日期：2023年12...,detailed,5,问题询问的是具体事件的预测，是实时性的，而答案直接给出了对美元指数走势的判断，不是简单的文本描述。,train
11,41d95288-441d-4c02-948a-6a3f0f4ef3ba,2023年全球货物贸易量指数和价格指数有何变动趋势？,下行,全球货物贸易量指数和价格指数下行，主要经济体出口贸易同比增速下降。,全球经济金融展望报告\n中国银行研究院 1 2024年\n全球经济复苏疲软，货币政策取向分化...,detailed,5,问题询问的是未来的趋势，需要分析或预测，不是简单的事实复述。答案直接给出了预测结果，符合要求。,train
12,41d95288-441d-4c02-948a-6a3f0f4ef3ba,2023年欧美央行的货币政策趋势是什么？,延续收紧态势，但步伐整体放缓,欧美央行货币政策延续收紧态势，但步伐整体放缓。,全球经济金融展望报告\n中国银行研究院 1 2024年\n全球经济复苏疲软，货币政策取向分化...,detailed,5,问题询问的是实时的经济政策趋势，具有明确的针对性和价值。答案直接给出了具体的方向和特点，不依...,train


In [10]:
qa_df['question'].nunique()

289

In [11]:
qa_df = qa_df.drop_duplicates('question')

In [12]:
len(qa_df)

289

In [13]:
qa_df.isnull().sum()

uuid             0
question         0
answer           0
context         48
doc              0
qa_type          0
score            0
score_reason     0
dataset          0
dtype: int64

In [14]:
def build_qa_samples(df, neg_batch_size=-1, n_neg_batch=5):
    """
    构建qa样本
    :param df: 包含qa的DataFrame，共两列，question和answer
    :param neg_batch_size: 负样本数量，为-1时表示将所有负样本和单个正样本配对，否则会将负样本拆开，结果中的query可能会重复
    """
    from tqdm.auto import tqdm
    import math

    data = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        question = row['question']
        answer = row['answer']
        # 筛选同category的，增加难度
        neg_samples = df[df['question'] != question]['answer'].values.tolist()
        neg_batch_count = math.ceil((len(df) - 1) / neg_batch_size)
        neg_batch_count = min(n_neg_batch, neg_batch_count)
        for neg_batch_idx in range(neg_batch_count):
            batch_neg_samples = neg_samples[neg_batch_idx * neg_batch_size: (neg_batch_idx + 1) * neg_batch_size]
            batch_neg_samples = [item for item in batch_neg_samples if item != answer]
            data.append({
                'query': question,
                'pos': [answer],
                'neg': batch_neg_samples
            })
    return data

def write_samples(samples, save_filename):
    import json

    with open(save_filename, 'w') as f:
        for sample in samples:
            f.write(json.dumps(sample, ensure_ascii=False))
            f.write('\n')

In [15]:
len(qa_df)

289

In [20]:
qa_df = qa_df[qa_df['qa_type'] == 'detailed']
qa_df['answer'] = qa_df['context']

qd_samples = build_qa_samples(qa_df, neg_batch_size=16, n_neg_batch=32)

  0%|          | 0/241 [00:00<?, ?it/s]

In [21]:
len(qd_samples)

3615

In [22]:
qd_samples[0]

{'query': '美元指数在2023年的走势如何？',
 'pos': ['美元指数高位震荡后走弱'],
 'neg': ['全球货物贸易量指数和价格指数下行，主要经济体出口贸易同比增速下降。',
  '欧美央行货币政策延续收紧态势，但步伐整体放缓。',
  '高利率环境抑制债券融资需求，债券违约风险持续上升，美国政府债务可持续性问题引发市场关注。',
  '展望2024年，预计全球经济复苏将依旧疲软，主要经济体增长态势和货币政策将进一步分化。',
  '发达经济体增速明显放缓，预计2023年增速较2022年下降1个百分点。',
  '新兴经济体增速与2022年大致持平，预计2023年增速比2022年下降0.1个百分点。',
  '美国GDP环比增长折年率为4.9%，比二季度增速高2.8个百分点。',
  '其中，主要新兴经济体工业生产指数普遍走高，如俄罗斯、土耳其、南非等，而发达经济体中的美国和韩国回升，英国、德国、意大利下行，日本波动较大，整体趋于平稳。',
  '主要新兴经济体工业生产指数普遍走高，如俄罗斯、土耳其、南非等',
  '全球融资环境收紧和经济下行压力对工业生产前景带来较大影响',
  '欧洲各国消费指数整体维持稳定（图4）',
  '是上半年免于陷入衰退的主要动力。',
  'OECD消费者信心指数从7月开始连续3个月回落',
  '美国私人投资在2023年一季度触底后逐渐反弹',
  '2023年二季度，欧元区固定资本形成总额环比增长0.1%，比一季度增速下降0.3个百分点',
  '房地产对GDP环比增长拉动率转为负值']}

In [23]:
write_samples(qd_samples, os.path.join(output_dir, 'emb_samples_qd_v2.jsonl'))